In [1]:
from collections import Counter
import numpy as np
import codecs
import re
import jieba

In [2]:
##　准备数据
import jieba
import pandas as pd 

# df_technology
df_technology  = pd.read_csv("./data/technology_news.csv", encoding = 'utf-8')
df_technology  = df_technology.dropna() # 取消可能为空的行数
# sprots
df_sprots   = pd.read_csv("./data/sports_news.csv", encoding = 'utf-8')
df_sprots   = df_sprots.dropna()

#df_society
df_society = pd.read_csv("./data/society_news.csv",encoding = 'utf-8')
df_society = df_society.dropna()

#military
df_military   = pd.read_csv("./data/military_news.csv", encoding = 'utf-8')
df_military   = df_military.dropna()

df_internality = pd.read_csv("./data/international_news.csv",encoding = 'utf-8')
df_international = df_internality.dropna()

df_house = pd.read_csv("./data/house_news.csv",encoding = 'utf-8')
df_house = df_house.dropna()

df_home = pd.read_csv("./data/home_news.csv",encoding = 'utf-8')
df_home = df_home.dropna()

df_finance = pd.read_csv("./data/finance_news.csv",encoding = 'utf-8')
df_finance = df_finance.dropna()

# df_entertainment 
df_entertainment   = pd.read_csv("./data/entertainment_news.csv", encoding = 'utf-8')
df_entertainment   = df_entertainment.dropna()

# df_car
df_car  = pd.read_csv("./data/car_news.csv", encoding = 'utf-8')
df_car  = df_car.dropna()

In [3]:
# 每个种类提取一定数量的文本样本记录
technology     = df_technology.content.values.tolist()
car            = df_car.content.values.tolist()
sprots         = df_sprots.content.values.tolist()
military       = df_military.content.values.tolist()
society        = df_society.content.values.tolist()
international  = df_international.content.values.tolist()
house          = df_house.content.values.tolist()
home           = df_home.content.values.tolist()
finance        = df_finance.content.values.tolist()
entertainment  = df_entertainment.content.values.tolist()

###  1.1统计文本长度

In [8]:
AllTexts = [technology,sprots,society,military,international,house,home,car,finance,entertainment]
AllTexts_length = []
for text in AllTexts:
    count = 0
    for oneExample in text:
        if len(oneExample)>=100:
            count = count +1
    AllTexts_length.append(count)
#AllTexts_length.sort()
AllTexts_length

[13807, 15266, 104606, 6508, 26947, 7096, 66151, 6282, 68893, 20021]

In [63]:
9000*0.7,9000*0.29,9000*0.01

(6300.0, 2610.0, 90.0)

### 1.2手动切分数据

In [32]:
allcontents = []
def Train_Test_Validation_Data_Split(filetrain,filetest,fileval,contents,labels,limits,lens):
    data_train = open(filetrain, 'a', encoding='utf-8')
    data_test  = open(filetest, 'a', encoding='utf-8')
    data_val   = open(fileval, 'a', encoding='utf-8')
    count = 0
    for content in contents:
        if len(content)>=limits:
            count = count + 1
            if count<lens:
                data_train.write(labels+'\t'+content+'\n')
            elif lens<=count and count<lens*1.1:
                data_test.write(labels+'\t'+content+'\n')
            elif lens*1.1<=count and count<lens*1.148:
                data_val.write(labels+'\t'+content+'\n')
            else:
                pass

In [33]:
limits = 50  #抽取超过特定长度的文本 
lens  =  8000 #每个样本抽取8000个样本
#AllTexts = [technology,sprots,society,military,international,house,home,car,finance,entertainment]
#labels   = ['技术','体育','社会','军事','国际','房产','家庭','汽车','金融','娱乐']
AllTexts = [military,international,house,home,car,finance,entertainment,technology,society,sprots]
labels   = ['军事','国际','房产','家庭','汽车','金融','娱乐','技术','社会','体育']
filetrain = './newdata/cnews_train.txt'
filetest  = './newdata/cnews_test.txt'
fileval   = './newdata/cnews_val.txt'
for i in range(0,len(AllTexts)):
    Train_Test_Validation_Data_Split(filetrain,filetest,fileval,AllTexts[i],labels[i],limits,lens)

### 1.3判断文件中的一行是不是为空，为空直接删掉

In [34]:
def del_blank(file1,file2):
    file1 = open(file1, 'r', encoding='utf-8') # 要去掉空行的文件 
    file2 = open(file2, 'w', encoding='utf-8') # 生成没有空行的文件
    try:
        for line in file1.readlines():
            if line == '\n':
                line = line.strip("\n")
            file2.write(line)
    finally:
        file1.close()
        file2.close()

In [ ]:
# 原始数据
filetrain = './newdata/cnews_train.txt'
filetest  = './newdata/cnews_test.txt'
fileval   = './newdata/cnews_val.txt'

# 清理空行后的数据
filetrain1 = './newdata/cnews.train.txt'
filetest1  = './newdata/cnews.test.txt'
fileval1   = './newdata/cnews.val.txt'

del_blank(filetrain,filetrain1)
del_blank(filetest,filetest1)
del_blank(fileval,fileval1)

### 1.4判断文件中的一行是不是为空，为空直接删掉；同时将样本乱序

In [56]:
import random
def delandrandom(file1,file2):
    contents = []
    file1 = open(file1, 'r', encoding='utf-8') # 要去掉空行的文件 
    file2 = open(file2, 'w', encoding='utf-8') # 生成没有空行的文件
    try:
        for line in file1.readlines():
            if line == '\n':
                line = line.strip("\n") #如果是空行直接删掉空行
                file2.write(line)
            else:
                contents.append(line)
            
        
        random.shuffle(contents)
        for content in contents:
            file2.write(content)
        
    finally:
        file1.close()
        file2.close()

In [57]:
# 原始数据
filetrain = './newdata/cnews_train.txt'
filetest  = './newdata/cnews_test.txt'
fileval   = './newdata/cnews_val.txt'

# 清理空行后的数据
filetrain1 = './newdata/random/cnews.train.txt'
filetest1  = './newdata/random/cnews.test.txt'
fileval1   = './newdata/random/cnews.val.txt'

delandrandom(filetrain,filetrain1)
delandrandom(filetest,filetest1)
delandrandom(fileval,fileval1)

### 基于样本收集词汇表和定制词嵌入矩阵###

In [36]:
from collections import Counter
import tensorflow.contrib.keras as kr
import numpy as np
import codecs
import re
import jieba

def read_file(filename):
    re_han = re.compile(u"([\u4E00-\u9FD5a-zA-Z0-9+#&\._%]+)")  # the method of cutting text by punctuation
    contents, labels = [], []
    with codecs.open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                line = line.rstrip()
                assert len(line.split('\t')) == 2
                label, content = line.split('\t')
                labels.append(label)
                blocks = re_han.split(content)
                word = []
                for blk in blocks:
                    if re_han.match(blk):
                        word.extend(jieba.lcut(blk))
                contents.append(word)
            except:
                pass
    return labels, contents
# labels[322] = '体育'
# np.array(contents[322]) = ['罗伊吃', '消炎药', '对抗', '膝伤', '开拓者',......]

In [47]:
def built_vocab_vector(filenames,voc_size=12000):
    '''
    去停用词，得到前9999个词，获取对应的词 以及 词向量
    :param filenames:
    :param voc_size:
    :return:
    '''
    stopword = open('./data/stopwords_NLP.txt', 'r', encoding='utf-8')
    stop = [key.strip(' \n') for key in stopword]
    all_data = []
    j = 1
    embeddings = np.zeros([11761, 100])  # 9656 是有效样本长度大小，仅仅需要更改这个即可，不许更改voc_size
    for filename in filenames:
        labels, content = read_file(filename)
        for eachline in content:
            line =[]
            for i in range(len(eachline)):
                if str(eachline[i]) not in stop:#去停用词
                    line.append(eachline[i])
            all_data.extend(line)
    counter = Counter(all_data)
    count_paris = counter.most_common(voc_size-1)
    word, _ = list(zip(*count_paris))
    f = codecs.open('./newdata/vector_word.txt', 'r', encoding='utf-8')
    vocab_word = open('./newdata/vocab_word.txt', 'w', encoding='utf-8')
    for ealine in f:
        item = ealine.split(' ')
        key = item[0]
        vec = np.array(item[1:], dtype='float32')
        if key in word:
            embeddings[j] = np.array(vec)
            vocab_word.write(key.strip('\r') + '\n')
            j += 1
    
    print("embeddings 填充行数是:",j)
    np.savez_compressed('./newdata/vector_word.npz', embeddings=embeddings)

In [48]:
# 清理空行后的数据
train_filename = './newdata/cnews.train.txt'
test_filename  = './newdata/cnews.test.txt'
val_filename   = './newdata/cnews.val.txt'

In [49]:
filename = [train_filename,test_filename,val_filename] # input is one list

In [50]:
import time
t1 = time.time()
built_vocab_vector(filename)
t2 = time.time()
print("运行时间%0.2f:"%((t2 - t1)/60),'mins')

embeddings 填充行数是: 11761
